+ Requirements:
    1. The "tools_camera.py" file.
    2. Images of the "imgs/calibration" and "imgs/calibration_wide" directories for the example calibration with OpenCV functionality.
    3. The estimated time to carry out this activity is approximately half an hour.
<br><br>
+ Objectives of the notebook:
    1. Calibrate the model of a camera.
<br><br>
+ Summary of activities:
    1. Code implemented in Python with Numpy will be analyzed.
    2. Code implemented with OpenCV functionality will be analyzed.

# Camera calibration

The calibration process consists of estimating the projection matrix $^{d}\mathbf{M}_w$ of a camera. We know that this matrix allows us to calculate the undistorted image pixels $\mathbf{p}_{d}$ from the world points $\mathbf{p}_w$.

$$ \mathbf{p}_{d} = \begin{bmatrix} s \, u \\ s \, v \\ s \end{bmatrix} ;\:\:\:\: \mathbf{p}_ {w} = \begin{bmatrix} {X}_w \\ {Y}_w \\ {Z}_w \\ 1 \end{bmatrix} ;\:\:\:\: \mathbf{p}_{d } = \, ^{d}\mathbf{M}_w \cdot \mathbf{p}_w$$

$$\begin{bmatrix} s \, u \\ s \, v \\ s \end{bmatrix} = \begin{bmatrix} {m}_{11}&{m}_{12}&{m} 13}&{m}_{14} \\ {m}_{21}&{m}_{22}&{m}_{23}&{m}_{24} \\ {m} 31}&{m}_{32}&{m}_{33}&{m}_{34} \end{bmatrix} \cdot \begin{bmatrix} {X}_w \\ {Y}_w \\ {Z}_w \\ 1 \end{bmatrix} $$

Calibrating a camera consists of solving the following system of equations [(Faugueras)](https://polibuscador.upv.es/permalink/f/6lh1qt/alma2139627090003706), based on $n$ (at least 6) correspondences between non-coplanar world points $\mathbf{p}_w$ with their respective pixels projected onto the image $\mathbf{p}_{d}$.

$$ \mathbf{A} \cdot \mathbf{m} = \mathbf{0} $$

$$\begin{bmatrix} 
{X}_1 & {Y}_1 & {Z}_1 & 1 & 0 & 0 & 0 & 0 & -{u}_1\,{X}_1 & -{u}_1\,{Y}_1 & -{u}_1\,{Z}_1 & -{u}_1 \\
0 & 0 & 0 & 0 & {X}_1 & {Y}_1 & {Z}_1 & 1 & -{v}_1\,{X}_1 & -{v}_1\,{Y}_1 & -{v}_1\,{Z}_1 & -{v}_1 \\
.. & .. & .. & .. & .. & .. & .. & .. & .. & .. & .. & .. \\
{X}_n & {Y}_n & {Z}_n & 1 & 0 & 0 & 0 & 0 & -{u}_n\,{X}_n & -{u}_n\,{Y}_n & -{u}_n\,{Z}_n & -{u}_n \\
0 & 0 & 0 & 0 & {X}_n & {Y}_n & {Z}_n & 1 & -{v}_n\,{X}_n & -{v}_n\,{Y}_n & -{v}_n\,{Z}_n & -{v}_n \\
\end{bmatrix} \cdot
\begin{bmatrix}
{m}_{11} \\ {m}_{12} \\ .. \\ {m}_{33} \\ {m}_{34}
\end{bmatrix}
= \mathbf{0} \\
$$

$$ {m}_{34} = 1 \:\:\: \implies \:\:\: \mathbf{A^{'}} \cdot \mathbf{m^{'}} = \mathbf{b} \:\:\: \implies \:\:\: \mathbf{m^{'}} = (\mathbf{A^{'T}} \mathbf{A^{'}})^{-1} \mathbf{A^{'T}} \, \, \mathbf{b} $$

# Calibration example in Python

In [ ]:
import numpy as np
import cv2
import os
from PIL import Image
from torchvision.transforms import transforms
from math import pi
import math
import random
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import clear_output, Image, display, HTML  # functionality to display images in the notebook
import tools_camera as ca
import glob
import pickle
import yaml

In [ ]:
"""World points"""
nPuntos = 6
amplitudX = 500
amplitudY = 500
amplitudZ = 500

nPuntos_mitad = int(nPuntos/2)
nPuntos_resto = int(nPuntos - nPuntos_mitad)

x = (np.random.rand(1, nPuntos) - 0.5) * amplitudX
y = np.concatenate( ( np.random.rand(1, nPuntos_mitad) * amplitudY, np.zeros((1, nPuntos_resto)) ), axis=1 )
z = np.concatenate( ( np.zeros((1, nPuntos_mitad)), np.random.rand(1, nPuntos_resto) * amplitudZ ), axis=1 )
X =  np.concatenate( (x,y,z,np.ones((1,nPuntos))), axis=0)

if nPuntos == 6:
    print('"World points:')
    print(ca.ndtotext(X))

In [ ]:
"""Intrinsec parameters"""
fu = 1580  # fu = f/dx
fv = 1580  # fv = f/dy
uo = 350  # uo = xo
vo = 300  # vo = yo

"""Extrinsec parameters"""
radio = 1000
angulo = 180 - 45

"""CAMERA MODEL"""
cam = ca.Cam(fu, fv, uo, vo, radio, angulo)

In [ ]:
"""Image points (in pixels)"""
U=cam.project(X)

if nPuntos == 6:
    print('Image points (in pixels):')
    print(ca.ndtotext(U))

In [ ]:
"""Noise introduction in coordinates points"""
ruidoM = 0.1 # Noise added to world points
ruidoI = 0.5 # Noise added to image points

Xr = X + np.concatenate((np.array((np.random.rand(3, nPuntos) - 0.5) * ruidoM), np.zeros((1, nPuntos))))  # mundo
Ur = U + np.concatenate((np.array((np.random.rand(2, nPuntos) - 0.5) * ruidoI), np.zeros((1, nPuntos))))  # imagen

if nPuntos == 6:
    print('Noisy world points:')
    #np.set_printoptions(precision=4)
    print(ca.ndtotext(Xr))
    print('Noisy image points:')
    print(ca.ndtotext(Ur))

In [ ]:
"""Data normalization"""
[Xn, Tx] = ca.normal(Xr)
[Un, Tu] = ca.normal(Ur)

if nPuntos == 6:
    print('Normalized projected point (in pixels):')
    print(ca.ndtotext(Un))

In [ ]:
"""Camera calibration with normalized points"""
PestN, covarzaN = ca.minEst(Xn, Un)

In [ ]:
"""Un-normalize without taking into account the variance"""
PestN1 = np.transpose(PestN)
PestN1.resize(3,4)
Pest1 = np.dot(np.dot(np.linalg.inv(Tu), PestN1), Tx)

Pest1=Pest1/Pest1[2][3]
print('Calibrated projection matrix:')
print(ca.ndtotext(Pest1))

In [ ]:
"""Un-normalize taking into account the variance"""
Tu = np.linalg.inv(Tu)
Tu = np.concatenate(
    (np.concatenate((Tu[0][0] * np.identity(4), Tu[0][1] * np.identity(4), Tu[0][2] * np.identity(4)), axis=1),
     np.concatenate((Tu[1][0] * np.identity(4), Tu[1][1] * np.identity(4), Tu[1][2] * np.identity(4)), axis=1),
     np.concatenate((Tu[2][0] * np.identity(4), Tu[2][1] * np.identity(4), Tu[2][2] * np.identity(4)), axis=1)))
Tx = np.transpose(Tx)
Tx = np.concatenate((np.concatenate((Tx, np.zeros((4, 8))), axis=1),
                     np.concatenate((np.zeros((4, 4)), Tx, np.zeros((4, 4))), axis=1),
                     np.concatenate((np.zeros((4, 8)), Tx), axis=1)))

PestN.resize((12, 1))
Pnorm = np.dot(np.dot(Tx, Tu), PestN)
PnormV = np.dot(np.dot(np.dot(np.dot(Tx, Tu), covarzaN), np.transpose(Tu)), np.transpose(Tx))

"""Estimating parameters extracted from the calibrated projection matrix"""
Pm, Pvar = ca.param(Pnorm, PnormV)

#('fu','fv','u0','v0','tx','ty','tz')
print('Real parameters:')
cTw = cam.get_extrinsec_matrix()
real_values = np.array([[fu, fv, uo, vo, cTw[0,3], cTw[1,3], cTw[2,3]]])
print(ca.ndtotext(real_values))
print('Estimated parameters:')
print(ca.ndtotext(np.transpose(Pm)))
print('Estimated variances:')
print(ca.ndtotext(np.transpose(Pvar)))

In [ ]:
"""Retroprojection error estimation"""
Ur_est = np.dot(Pest1, Xr)
Ur_est = np.concatenate([[Ur_est[0] / Ur_est[2]], [Ur_est[1] / Ur_est[2]], [np.ones(Ur_est[0].shape)]])
resta = np.concatenate(([U[0] - Ur_est[0]], [U[1] - Ur_est[1]]))
dista = np.diag(np.dot(np.transpose(resta), resta))
error_retroproyeccion = np.mean(np.sqrt(dista))
        
print('error_retroproyeccion:', error_retroproyeccion)

In [ ]:
"""The image is displayed comparing the original pixels with the pixels projected with the calibrated matrix"""
fig = plt.figure(figsize=(8,6))
plt.plot(U[0][:nPuntos_mitad], U[1][:nPuntos_mitad], marker='o', ls='', color='k');
plt.plot(U[0][nPuntos_mitad:], U[1][nPuntos_mitad:], marker='o', ls='', color='k');
plt.plot(Ur_est[0], Ur_est[1], marker='.', ls='', color='r');

# Calibration example using OpenCV functionality
OpenCV uses the calibration method of [Zhang](http://ieeexplore.ieee.org/document/888718) which starts from several images of coplanar world points. This way the calibration pattern is much easier to build, since a pattern can simply be printed with a printer.

In [ ]:
""""Extraction of pairs of world points and their corresponding projected points in pixels"""
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*8,3), np.float32)
objp[:,:2] = np.mgrid[0:8, 0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('imgs/calibration_wide/GO*.jpg')
#images = glob.glob('imgs/calibration/left*.jpg')

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (8,6), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (8,6), corners, ret)
        #write_name = 'corners_found'+str(idx)+'.jpg'
        #cv2.imwrite(write_name, img)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

In [ ]:
"""Camera calibration"""
# Do camera calibration given object points and image points
img_size = (img.shape[1], img.shape[0])
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
#print('ret:', ret, 'mtx:', mtx, 'dist:', dist, 'rvecs:', rvecs, 'tvecs:', tvecs)

#S = cv2.calibrationMatrixValues(mtx, img_size, 6.17, 4.55);
#print(S)
#fovx	Output field of view in degrees along the horizontal sensor axis.
#fovy	Output field of view in degrees along the vertical sensor axis.
#focalLength	Focal length of the lens in mm.
#principalPoint	Principal point in mm.
#aspectRatio	fy/fx

# Show P of the first calibration image
K = mtx
R = cv2.Rodrigues(rvecs[0])[0]
T = tvecs[0]
RT = np.concatenate((R,T),axis=1)
Pest0 = np.dot(K, RT)

#print('Matriz de intrínsecos:')
#print(ca.ndtotext(K))

#print('Matriz de rotación:')
#print(ca.ndtotext(R))

#print('Vector de traslación:')
#print(ca.ndtotext(T)

print('Projection matrix for image0:')
print(ca.ndtotext(Pest0))

In [ ]:
"""Save the camera calibration result for later use (we won't worry about rvecs / tvecs)"""
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "imgs/calibration_wide/wide_dist_pickle.p", "wb" ) )

In [ ]:
"""Show error in the first image with OpenCV funcionality"""
U = cv2.projectPoints(objpoints[0], rvecs[0], tvecs[0], mtx, dist)
error = np.asarray(U[0][:,0]-imgpoints[0][:,0,:])
error = np.linalg.norm(error)/len(error)
print("Retroprojection error with OpcenCV for first image:", error)

# Show error in the first image with Python funcionality
mpoints = np.transpose(objpoints[0])
n_points = len(mpoints[0])
lista_ones = np.ones((1,n_points)).tolist()
mpoints =  np.append(mpoints, lista_ones, axis=0)

U = np.dot(Pest0, mpoints)
U = [ [U[0] / U[2]], [U[1] / U[2]] ]
U = np.reshape(np.array(U, dtype="float32").transpose(), (n_points,1,2))

imgpoints_nd = cv2.undistortPoints(imgpoints[0], mtx, dist, None, mtx)
error1 = U-imgpoints_nd
error1 = np.linalg.norm(error1)/len(error1)
#error1 = cv2.norm(U, imgpoints_nd, cv2.NORM_L2)/len(imgpoints_nd)
print("Retroprojection error for first image:",error1)

In [ ]:
""""Display an undistorted Test image"""
img = cv2.imread('imgs/calibration_wide/test_image.jpg')
dst = cv2.undistort(img, mtx, dist, None, mtx)
#dst = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
#cv2.imwrite('imgs/calibration_wide/test_undist.jpg',dst)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)